In [195]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import nltk
from nltk.corpus import stopwords

In [196]:
stop=stopwords.words('english')

In [197]:
# read in data
dem = pd.read_parquet("/Users/erikafox/Desktop/NLP_final_project/10_datasets/democrats.parquet", engine='fastparquet')
rep = pd.read_parquet("/Users/erikafox/Desktop/NLP_final_project/10_datasets/republican.parquet", engine='fastparquet')
neut = pd.read_parquet("/Users/erikafox/Desktop/NLP_final_project/10_datasets/neutral.parquet", engine='fastparquet')

In [198]:
# add type to each datafram
dem['label_type']="democrat"
rep['label_type']="republican"
neut['label_type']="neutral"

In [199]:
# combine data
data=dem.append(rep)
data=data.append(neut)
data['label_type'].value_counts()

democrat      4937
republican    4809
neutral       4760
Name: label_type, dtype: int64

In [200]:
dem["total_post"]

index
4279    It’s stuff like this that makes me concerned a...
4770    This feels like a Republican talking point to ...
4781    Fair fair...all the more reason to vote, let's...
3459    you are evidently conservative, and if you are...
3345    Unbelievable. As the article says the pandemic...
                              ...                        
1739    As of Veterans Day… 56 bills have passed the h...
1740    How to respond to a friend saying that trump i...
1741    Drugmakers including Bristol-Myers Squibb, Gil...
1742    Children with diseases (including cancer) shou...
1743    The Democratic Nominee Could Use the Slogan "H...
Name: total_post, Length: 4937, dtype: object

In [201]:
train_data = data.sample(frac=0.8,random_state=1)
test_data = data.drop(train_data.index)

In [202]:
train_data["label_type"].value_counts()
test_data["label_type"].value_counts()

neutral       653
republican    506
democrat      382
Name: label_type, dtype: int64

In [203]:
dem=train_data[train_data["label_type"]=="democrat"]
rep=train_data[train_data["label_type"]=="republican"]
neut=train_data[train_data["label_type"]=="neutral"]

In [204]:
# step2: create term-document matrices

# dem
docs = dem["total_post"]
vec = CountVectorizer()
X = vec.fit_transform(docs)
demvec=vec.get_feature_names_out()
demtdm = pd.DataFrame(X.toarray(), columns=demvec)

# keep a version with full counts
demtdm2=demtdm


#for now we just need to know if its there or not. we can revisit this
for i in demtdm:
    demtdm[i]=np.where(demtdm[i]>0,1,0)


In [205]:
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

demdrop=[]
for i in demtdm:
    if has_numbers(i) or ("_" in i) or (i in stop):
        demdrop.append(i)
demtdm=demtdm.drop(columns=demdrop)


demtdm



,aa,abandon,abandoned,abandons,abatement,abbr,abc,abdicate,abduct,abductions,...,zero,ziegler,ziplock,zodiac,zombie,zombies,zone,zoomers,zte,über
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3956,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3958,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3959,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [206]:
#rep

docs = rep["total_post"]
vec = CountVectorizer()
X = vec.fit_transform(docs)
repvec=vec.get_feature_names_out()
reptdm = pd.DataFrame(X.toarray(), columns=repvec)
reptdm2=reptdm

for i in reptdm:
    reptdm[i]=np.where(reptdm[i]>0,1,0)


In [207]:
repdrop=[]
for i in reptdm:
    if has_numbers(i) or ("_" in i) or (i in stop):
        repdrop.append(i)
reptdm=reptdm.drop(columns=repdrop)

#reptdm

In [208]:
# neut

docs = neut["total_post"]
vec = CountVectorizer()
X = vec.fit_transform(docs)
neutvec=vec.get_feature_names_out()
neuttdm = pd.DataFrame(X.toarray(), columns=neutvec)
neuttdm2=neuttdm

for i in neuttdm:
    neuttdm[i]=np.where(neuttdm[i]>0,1,0)


In [209]:
neutdrop=[]
for i in neuttdm:
    if has_numbers(i) or ("_" in i)or (i in stop):
        neutdrop.append(i)
neuttdm=neuttdm.drop(columns=neutdrop)

In [210]:
#tdm for full trainset

docs = train_data["total_post"]
vec = CountVectorizer()
X = vec.fit_transform(docs)
datvec=vec.get_feature_names_out()
tdm = pd.DataFrame(X.toarray(), columns=datvec)

drop=[]
for i in tdm:
    if has_numbers(i) or ("_" in i) or (i in stop):
        drop.append(i)
tdm=tdm.drop(columns=drop)

tdm




,aa,aave,ab,abacuses,abandon,abandoned,abandoning,abandons,abatement,abbott,...,zoologically,zoom,zoomers,zte,zucht,zuckerberg,über,ˈjerēˌmandər,ﾟヮﾟ,𝓷𝓲𝓬𝓮
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11600,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11601,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11602,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11603,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [211]:
train_data = pd.concat([train_data.reset_index(), tdm], axis=1).iloc[:,1:]


In [212]:
train_data=train_data.drop(columns=["id","title","subreddit","score","text","type"])

In [213]:
train_data["total_post"]=train_data["total_post"].str.lower()

In [214]:
#step 3: compute frequencies


#dem

demfreq=pd.DataFrame(demvec)

demfreq=demfreq.rename(columns={0: "terms"})


demfreq = demfreq[~demfreq['terms'].isin(demdrop)]


demfreq['frequency'] = demfreq.apply(
    lambda row: demtdm[row.terms].sum(), axis=1)


demfreq.sample(10)


demfreq['occurence'] = demfreq.apply(
    lambda row: row.frequency/3961, axis=1)

demfreq.sort_values(by="occurence",ascending=False).head(50)

,terms,frequency,occurence
11670,trump,1120,0.282757
8306,people,696,0.175713
6738,like,554,0.139864
12574,would,493,0.124464
5016,get,460,0.116132
1411,biden,459,0.115880
12201,vote,429,0.108306
11378,think,428,0.108054
7908,one,391,0.098712
9710,right,327,0.082555


In [215]:
#rep

repfreq=pd.DataFrame(repvec)

repfreq=repfreq.rename(columns={0: "terms"})


repfreq = repfreq[~repfreq['terms'].isin(repdrop)]


repfreq['frequency'] = repfreq.apply(
    lambda row: reptdm[row.terms].sum(), axis=1)


repfreq.sample(10)


repfreq['occurence'] = repfreq.apply(
    lambda row: row.frequency/3837, axis=1)

repfreq.sort_values(by="occurence",ascending=False).head(50)



,terms,frequency,occurence
8330,people,853,0.222309
6782,like,676,0.176179
11743,trump,526,0.137086
12651,would,513,0.133698
11425,think,450,0.117279
7944,one,409,0.106594
5027,get,406,0.105812
4165,even,381,0.099296
6517,know,330,0.086005
9712,right,328,0.085483


In [216]:
#neut

neutfreq=pd.DataFrame(neutvec)

neutfreq=neutfreq.rename(columns={0: "terms"})


neutfreq = neutfreq[~neutfreq['terms'].isin(neutdrop)]

neutfreq['frequency'] = neutfreq.apply(
    lambda row: neuttdm[row.terms].sum(), axis=1)


neutfreq.sample(10)


neutfreq['occurence'] = neutfreq.apply(
    lambda row: row.frequency/3807, axis=1)

neutfreq.sort_values(by="occurence",ascending=False).head(50)

neutfreq.shape[0]

12535

In [217]:
Ndem = demfreq.shape[0]
Nrep = repfreq.shape[0]
Nneut = neutfreq.shape[0]

In [218]:
Nvoc = len(train_data.columns)
alpha=1

In [219]:
train_data.columns

Index(['total_post', 'label_type', 'aa', 'aave', 'ab', 'abacuses', 'abandon',
       'abandoned', 'abandoning', 'abandons',
       ...
       'zoologically', 'zoom', 'zoomers', 'zte', 'zucht', 'zuckerberg', 'über',
       'ˈjerēˌmandər', 'ﾟヮﾟ', '𝓷𝓲𝓬𝓮'],
      dtype='object', length=20943)

In [220]:
def p_w_dem(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['label_type'] == 'democrat', word].sum() + alpha) / (Ndem + alpha*Nvoc)
    else:
        return 1
      
def p_w_rep(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['label_type'] == 'republican', word].sum() + alpha) / (Nrep + alpha*Nvoc)
    else:
        return 1

def p_w_neut(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['label_type'] == 'neutral', word].sum() + alpha) / (Nneut + alpha*Nvoc)
    else:
        return 1

In [221]:
def classify(message):


    Pdem = train_data["label_type"].value_counts()["democrat"] / train_data.shape[0]
    Prep = train_data["label_type"].value_counts()["republican"] / train_data.shape[0]
    Pneut = train_data["label_type"].value_counts()["neutral"] / train_data.shape[0]

    for word in message.split():


        Pdem *= p_w_dem(word)
        Prep *= p_w_rep(word)
        Pneut *= p_w_neut(word)


    

    ans = max([Pdem,Pneut,Prep])
    
    if ans == Pdem:
        return "democrat"
    elif ans == Prep:
        return "republican"
    else:
        return "neutral"

In [222]:
test_data['predicted'] = test_data.apply(
    lambda row: classify(row.total_post), axis=1)

/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_48769/1901069398.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted'] = test_data.apply(


In [225]:
test_data

test_data["correct"]=np.where(test_data["label_type"]==test_data["predicted"],True,False)

test_data["correct"].value_counts()


/var/folders/5b/sjkc4b8n0hs41zd03hxr66100000gn/T/ipykernel_48769/3693436028.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["correct"]=np.where(test_data["label_type"]==test_data["predicted"],True,False)


True     929
False    612
Name: correct, dtype: int64

In [226]:
929/(929+616)

0.601294498381877